In [2]:
import pyzx as zx
import fractions

state = zx.Graph()
out1 = state.add_vertex(zx.VertexType.BOUNDARY, 0, 1)
out2 = state.add_vertex(zx.VertexType.BOUNDARY, 1, 1)
in1 = state.add_vertex(zx.VertexType.X, 0, 0)
in2 = state.add_vertex(zx.VertexType.X, 1, 0)
state.add_edges([(in1, out1), (in2, out2)])
state.set_outputs([out1, out2])
state.set_inputs([])

circ1 = zx.qasm("""
qreg qubits[2];

h qubits[0];
cx qubits[0], qubits[1];
s qubits[0];
""").to_graph()

circ2 = zx.qasm("""
qreg qubits[2];

h qubits[0];
cx qubits[0], qubits[1];
s qubits[0];
z qubits[0];
x qubits[0];
x qubits[1];
""").to_graph()

x = zx.Scalar()
x.phase = fractions.Fraction(1, 2)
circ2.scalar.mult_with_scalar(x)

zx.draw(circ1 * state)
zx.draw(circ2 * state)

display(zx.compare_tensors(circ1 * state, circ2 * state, preserve_scalar=True))

display(zx.print_matrix(circ1 * state))
display(zx.print_matrix(circ2 * state))

True

Label(value='\\begin{equation}\n\\sqrt{2}\n\\begin{pmatrix}\n1 \\\\ \n0 \\\\ \n0 \\\\ \ni\n\\end{pmatrix}\n\\e…

Label(value='\\begin{equation}\n\\sqrt{2}\n\\begin{pmatrix}\n1 \\\\ \n0 \\\\ \n0 \\\\ \ni\n\\end{pmatrix}\n\\e…

In [3]:
def id(len: int):
    return zx.qasm(f"qreg x[{len}];").to_graph()

def cnot(len: int, control: int, target: int):
    return zx.qasm(f"qreg x[{len}]; cx x[{control}], x[{target}];").to_graph()

def had(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; h x[{qubit}];").to_graph()

def t(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; t x[{qubit}];").to_graph()

def tdg(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; tdg x[{qubit}];").to_graph()

def s(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; s x[{qubit}];").to_graph()

def sdg(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; sdg x[{qubit}];").to_graph()

def rootx(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; h x[{qubit}]; s x[{qubit}]; h x[{qubit}];").to_graph()

def rootxdg(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; h x[{qubit}]; sdg x[{qubit}]; h x[{qubit}];").to_graph()

def z(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; z x[{qubit}];").to_graph()

def x(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; x x[{qubit}];").to_graph()
 

def create_pauli(string: str, sign: bool = True):
    base = id(len(string))

    for i in range(len(string)):
        if string[i] == "X":
            base = had(len(string), i) * base
        elif string[i] == "Y":
            base = rootx(len(string), i).adjoint() * base

    
    base = create_phase_poly(string.replace("X", "Z").replace("Y", "Z"), sign) * base

    for i in range(len(string)):
        if string[i] == "X":
            base = had(len(string), i) * base
        elif string[i] == "Y":
            base = rootx(len(string), i) * base

    return base


def create_phase_poly(string: str, sign: bool = True, i: int = 0):
    if i >= len(string):
        return id(len(string))
    rec = create_phase_poly(string, sign, i + 1)
    if string[i] == "I":
        return rec
    next_i = i + 1
    while next_i < len(string) and string[next_i] == "I":
        next_i = next_i + 1
    if next_i >= len(string):
        if sign:
            return t(len(string), i)
        else:
            return tdg(len(string), i)
    return cnot(len(string), i, next_i) * rec * cnot(len(string), i, next_i)

control = "Z"
string = "XZIYZ"

control = string.index(control)
target = 4

graph = cnot(5, control, target) * create_pauli(string)

simp_graph = create_pauli("XIIYZ") * cnot(5, control, target)

display(zx.print_matrix(graph))
display(zx.print_matrix(simp_graph))

display(zx.compare_tensors(graph, simp_graph))



    

Label(value='\\begin{equation}\n0.92e^{i\\frac{1}{8}\\pi}\n\\begin{pmatrix}\n1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0…

Label(value='\\begin{equation}\n0.92e^{i\\frac{1}{8}\\pi}\n\\begin{pmatrix}\n1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0…

True

In [4]:
pauli = create_pauli("ZY")
state1 = pauli * had(2, 0) * state
state2 = x(2, 0) * z(2, 1) * state1

zx.draw(state1)
zx.draw(state2)
display(zx.print_matrix(state1))

zx.print_matrix(state2)

Label(value='\\begin{equation}\n1.31e^{i\\frac{1}{8}\\pi}\n\\begin{pmatrix}\n1 \\\\ \n1-\\sqrt{2} \\\\ \n1 \\\…

Label(value='\\begin{equation}\n1.31e^{i\\frac{1}{8}\\pi}\n\\begin{pmatrix}\n1 \\\\ \n1-\\sqrt{2} \\\\ \n1 \\\…

In [ ]:
import math

def compare_tensors(ta, tb, verbose=False):
    epsilon = 1e-7
    tiny_eps = 1e-15
    min_dist = math.inf
    min_j = 0
    some = False
    for j,x in enumerate(ta.flat):
        max_dist = 0
        if abs(ta.flat[j]) < tiny_eps or abs(tb.flat[j]) < tiny_eps:
            continue
        some = True        
        difference = ta.flat[j] / tb.flat[j]
        for j2, x2 in enumerate(ta.flat):
            if j == j2: continue
            if abs(ta.flat[j2]) < tiny_eps or abs(tb.flat[j2]) < tiny_eps:
                dist = abs(ta.flat[j2] - tb.flat[j2])
            else:
                dist = abs(1 - difference/(x2/tb.flat[j2]))
            if dist > max_dist:
                max_dist = dist
        if max_dist < min_dist:
            min_j = j
            min_dist = max_dist
            if min_dist < epsilon:
                break

    if verbose:
        print(min_dist)
        print(ta.flat[min_j] / tb.flat[min_j])
        print(min_j)
        difference = ta.flat[min_j] / tb.flat[min_j]
        for j2,x2 in enumerate(ta.flat):
            if min_j == j2: continue
            if abs(ta.flat[j2]) < tiny_eps or abs(tb.flat[j2]) < tiny_eps: continue
            dist = abs(1 - difference/(x2/tb.flat[j2]))
            if dist > epsilon:
                print(x2, tb.flat[j2])
                print(dist)

    return some and min_dist < epsilon

def compare_circs(ga, gb, verbose=False):
    ta = ga.to_tensor(preserve_scalar=False)
    tb = gb.to_tensor(preserve_scalar=False)
    return compare_tensors


def print_tensor_pair(ta, tb):
    for j,x in enumerate(ta.flat):
        print(ta.flat[j], tb.flat[j])


In [72]:
def new_state(qubits: int):
  s = zx.Graph()
  outs = []
  for i in range(qubits):
    outs.append(s.add_vertex(zx.VertexType.BOUNDARY, i, 1))
    in1 = s.add_vertex(zx.VertexType.X, i, 0)
    s.add_edge((in1, outs[-1]))
  s.set_outputs(outs)
  s.set_inputs([])
  return s

file = open("test.qasm", "r+")
file2 = open("qasm_files/test.qasm", "r+")
lines = "\n".join(file.readlines())
lines2 = "\n".join(file2.readlines())

state15 = new_state(15)

a = zx.qasm(lines).to_graph()
b = zx.qasm(lines2).to_graph()

a = a * state15
b = b * state15

# zx.teleport_reduce(a)
# zx.teleport_reduce(b)


ta = a.to_tensor(preserve_scalar=False)
tb = b.to_tensor(preserve_scalar=False)



In [75]:
print(compare_tensors(ta, tb))
# print_tensor_pair(ta, tb)

True
